In [ ]:
#####Joint_hierarchical_structured_topic_model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
from scipy.special import psi as psi

#np.random.seed(98537)

In [ ]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    return z_id

In [ ]:
####データの生成####
##データの設定
#データのパラメータ
types = 2
k = 20
d = 10000 
s = np.random.poisson(5.0, d); s[s < 2] = 2
n = np.sum(s)
v1 = 1000
v2 = 500
a = 500
k_vec = np.repeat(1, k)

#文書idと単語頻度を生成
s_id = np.repeat(np.arange(d), s)
s_list = [i for i in range(d)]
for i in range(d):
    s_list[i] = np.array(np.where(s_id==i)[0], dtype="int")
w21 = np.random.poisson(np.random.gamma(15.0, 1/0.5, n)); w21[w21 < 5] = 5
w22 = np.random.poisson(np.random.gamma(7.5, 1/0.5, n)); w22[w22 < 5] = 5
w11 = np.array([np.sum(w21[s_list[i]]) for i in range(d)])
w12 = np.array([np.sum(w22[s_list[i]]) for i in range(d)])
f1 = np.sum(w11)
f2 = np.sum(w12)

#補助文書idを生成
x = np.random.poisson(15.0, d); x[x < 2] = 2
x_id = np.repeat(np.arange(d), x)
g = np.sum(x)

In [ ]:
##単語idとインデックスの設定
#単語idを設定
d_id11 = np.repeat(np.arange(d), w11)
d_id12 = np.repeat(np.arange(d), w12)
d_id21 = np.repeat(np.arange(n), w21)
d_id22 = np.repeat(np.arange(n), w22)

#インデックスを作成
d_list11 = [i for i in range(d)]; d_dt11 = [i for i in range(d)]
d_list12 = [i for i in range(d)]; d_dt12 = [i for i in range(d)]
d_list21 = [i for i in range(n)]; d_dt21 = [i for i in range(n)]
d_list22 = [i for i in range(n)]; d_dt22 = [i for i in range(n)]
for i in range(d):
    d_list11[i] = np.array(np.where(d_id11==i)[0], dtype="int")
    d_list12[i] = np.array(np.where(d_id12==i)[0], dtype="int")
    d_dt11[i] = np.repeat(1, w11[i])
    d_dt12[i] = np.repeat(1, w12[i])
for i in range(n):
    d_list21[i] = np.array(np.where(d_id21==i)[0], dtype="int")
    d_list22[i] = np.array(np.where(d_id22==i)[0], dtype="int")
    d_dt21[i] = np.repeat(1, w21[i])
    d_dt22[i] = np.repeat(1, w22[i])
    
#タグidを設定
a_id = np.repeat(np.arange(d), x)
a_list = [i for i in range(d)]
a_dt = [i for i in range(d)]
for i in range(d):
    a_list[i] = np.array(np.where(a_id==i), dtype="int")
    a_dt[i] = np.repeat(1, x[i])

In [ ]:
####応答変数を生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    ##パラメータを生成
    #トピック分布のパラメータを生成
    weights = 3.0
    theta = np.random.dirichlet(np.repeat(0.2, k), d)
    theta1 = np.array([np.random.dirichlet(weights*theta[s_id[i], ], 1).reshape(-1) for i in range(n)])
    theta2 = np.array([np.random.dirichlet(weights*theta[s_id[i], ], 1).reshape(-1) for i in range(n)])
    thetat = theta.copy(); thetat1 = theta1.copy(); thetat2 = theta2.copy()
    
    #単語分布のパラメータを生成
    phi1 = np.random.dirichlet(np.repeat(0.05, v1), k)
    phi2 = np.random.dirichlet(np.repeat(0.05, v2), k)
    omega = np.random.dirichlet(np.repeat(0.05, a), k)

    #出現確率が低い単語を入れ替える
    index = np.where(np.max(phi1, axis=0) <= (k*k)/f1)[0]
    for j in range(index.shape[0]):
        phi1[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index[j]] = (k*k)/f1
    index = np.where(np.max(phi2, axis=0) <= (k*k)/f2)[0]
    for j in range(index.shape[0]):
        phi2[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index[j]] = (k*k)/f2
    phit1 = phi1.copy(); phit2 = phi2.copy(); omegat = omega.copy()

    ##トピックから単語を生成
    #多項分布からトピックを生成
    Z1 = np.array(rmnom(theta1[d_id21, ], f1, k, np.arange(f1), 1)[1].todense(), dtype="int8")
    Z2 = np.array(rmnom(theta2[d_id22, ], f2, k, np.arange(f2), 1)[1].todense(), dtype="int8")
    z_vec1 = np.dot(Z1, np.arange(k))
    z_vec2 = np.dot(Z2, np.arange(k))

    #トピックから単語を生成
    word_data1 = np.zeros((f1, v1), dtype="int8")
    word_data2 = np.zeros((f2, v2), dtype="int8")
    word1 = np.repeat(0, f1)
    word2 = np.repeat(0, f2)
    for i in range(n):
        out1 = rmnom(phi1[z_vec1[d_list21[i]], ], w21[i], v1, np.arange(w21[i]), 1)
        out2 = rmnom(phi2[z_vec2[d_list22[i]], ], w22[i], v2, np.arange(w22[i]), 1)
        word_data1[d_list21[i], ] = np.array(out1[1].todense(), dtype="int8")
        word_data2[d_list22[i], ] = np.array(out2[1].todense(), dtype="int8")
        word1[d_list21[i]] = np.array(out1[0], dtype="int")
        word2[d_list22[i]] = np.array(out2[0], dtype="int")

    ##トピックからタグを生成
    #多項分布からトピックを生成
    gamma = np.array([np.sum(Z1[d_list11[i], ], axis=0) / w11[i] for i in range(d)])
    S = np.array(rmnom(gamma[a_id, ], g, k, np.arange(g), 1)[1].todense(), dtype="int8")
    s_vec = np.dot(S, np.arange(k))

    #トピックからタグを生成
    out = rmnom(omega[s_vec, ], g, a, np.arange(g), 1)
    tag_data = np.array(out[1].todense(), dtype="int8")
    tag = np.array(out[0], dtype="int")

    #break条件
    freq1 = np.min(np.sum(word_data1, axis=0)) 
    freq2 = np.min(np.sum(word_data2, axis=0)) 
    freq3 = np.min(np.sum(tag_data, axis=0)) 
    if (freq1 > 0) & (freq2 > 0) & (freq3 > 0):
        break

In [ ]:
##トピック分布間の関連性と単語分布の可視化
#トピック分布間のtf-idfを計算
normalize1 = np.sqrt(np.sum(np.power(theta[s_id, ], 2), axis=1)) * np.sqrt(np.sum(np.power(theta1, 2), axis=1))
inner_prod1 = np.sum(theta[s_id, ] * theta1, axis=1)
tf_idf1 = inner_prod1 / normalize1
normalize2 = np.sqrt(np.sum(np.power(theta[s_id, ], 2), axis=1)) * np.sqrt(np.sum(np.power(theta2, 2), axis=1))
inner_prod2 = np.sum(theta[s_id, ] * theta2, axis=1)
tf_idf2 = inner_prod2 / normalize2
normalize3 = np.sqrt(np.sum(np.power(theta1, 2), axis=1)) * np.sqrt(np.sum(np.power(theta2, 2), axis=1))
inner_prod3 = np.sum(theta1 * theta2, axis=1)
tf_idf3 = inner_prod3 / normalize3
del normalize1; normalize2; normalize3, inner_prod1, inner_prod2, inner_prod3

#tf-idfと単語分布を可視化
dt = [tf_idf1, tf_idf2, tf_idf3, np.max(phi1, axis=0), np.max(phi2, axis=0), np.max(omega, axis=0)]
fig_range = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, np.max(phi1), np.max(phi2), np.max(omega)]])
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
legend = ["グローバル分布と名詞分布のtf-idf分布", "グローバル分布と動詞分布のtf-idf分布", 
          "ローカル分布同士のtf-idf分布", "名詞分布のパラメータ", "動詞分布のパラメータ", "タグ分布のパラメータ"]
fig = plt.figure(figsize=(12.5, 6.5))
for j in range(len(dt)):
    ax = fig.add_subplot(2, 3, j+1)
    ax.hist(dt[j],  bins=25, range=(fig_range[0, j], fig_range[1, j]), color=colorlist[j])
    plt.title(legend[j], fontsize=12.5)
fig.tight_layout()
plt.show()

In [ ]:
####マルコフ連鎖モンテカルロ法でパラメータをサンプリング#####
#トピック尤度と負担率を計算する関数
def LLho(theta, phi, d_id, wd, f, k):
    Lho = theta[d_id, ] * (phi.T)[wd, ]
    topic_rate = Lho / np.sum(Lho, axis=1)[:, np.newaxis]
    return Lho, topic_rate

In [ ]:
##アルゴリズムの設定
R = 2000   #サンプリング回数
keep = 2   #2回に1回の割合でサンプリング結果を格納
disp = 10
iter = 0
burnin = int(500/keep)
no1 = np.arange(f1)
no2 = np.arange(f2)

In [ ]:
##単語とタグのインデックスを作成
#単語インデックスを作成
word_list1 = [i for i in range(v1)]; word_dt1 = [i for i in range(v1)]
word_list2 = [i for i in range(v2)]; word_dt2 = [i for i in range(v2)]
for i in range(v1):
    word_list1[i] = np.array(np.where(word1==i)[0], dtype="int")
    word_dt1[i] = np.repeat(1, word_list1[i].shape[0])
for i in range(v2):
    word_list2[i] = np.array(np.where(word2==i)[0], dtype="int")
    word_dt2[i] = np.repeat(1, word_list2[i].shape[0])
    
#タグインデックスを作成
tag_list = [i for i in range(a)]
tag_dt = [i for i in range(a)]
for i in range(a):
    tag_list[i] = np.array(np.where(tag==i)[0], dtype="int")
    tag_dt[i] = np.repeat(1, tag_list[i].shape[0])

In [ ]:
##事前分布の設定
alpha01 = 0.1
beta01 = 0.1
beta02 = 0.1

In [ ]:
##パラメータの真値
#ハイパーパラメータの初期値
Delta1 = np.array(np.full((n, k), 0.25), dtype="float32")
Delta2 = np.array(np.full((n, k), 0.25), dtype="float32")
delta_n1 = w21.copy(); delta_n1[delta_n1 < 5] = 5
delta_n2 = w22.copy(); delta_n2[delta_n2 < 5] = 5
er = 0.0001

#トピック分布の真値
theta = thetat.copy()
theta1 = thetat1.copy()
theta2 = thetat2.copy()

#単語分布のパラメータを生成
phi1 = phit1.copy()
phi2 = phit2.copy()
omega = omegat.copy()

In [ ]:
##初期値の設定
#ハイパーパラメータの初期値
Delta1 = np.array(np.full((n, k), 0.25), dtype="float32")
Delta2 = np.array(np.full((n, k), 0.25), dtype="float32")
delta_n1 = w21.copy(); delta_n1[delta_n1 < 5] = 5
delta_n2 = w22.copy(); delta_n2[delta_n2 < 5] = 5
er = 0.0001

#トピック分布の初期値
theta = np.random.dirichlet(np.repeat(2.5, k), d)
theta1 = np.random.dirichlet(np.repeat(2.5, k), n)
theta2 = np.random.dirichlet(np.repeat(2.5, k), n)

#単語分布の初期値
phi1 = np.random.dirichlet(np.repeat(2.5, v1), k)
phi2 = np.random.dirichlet(np.repeat(2.5, v2), k)
omega = np.random.dirichlet(np.repeat(2.5, a), k)

In [ ]:
##パラメータの格納用配列
#モデルパラメータの格納用配列
THETA = np.zeros((d, k, int(R/keep)))
THETA1 = np.zeros((n, k, int(R/keep)))
THETA2 = np.zeros((n, k, int(R/keep)))
PHI1 = np.zeros((k, v1, int(R/keep)))
PHI2 = np.zeros((k, v2, int(R/keep)))
OMEGA = np.zeros((k, a, int(R/keep)))

#トピックの格納用配列
SEG11 = np.zeros((f1, k))
SEG12 = np.zeros((f2, k))
SEG2 = np.zeros((g, k))

In [ ]:
##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst11 = np.sum(np.dot(word_data1, np.log(np.mean(word_data1, axis=0))))
LLst12 = np.sum(np.dot(word_data2, np.log(np.mean(word_data2, axis=0))))
LLst2 = np.sum(np.dot(tag_data, np.log(np.mean(tag_data, axis=0))))
LLst1 = LLst11 + LLst12
LLst = LLst1 + LLst2
print(np.round(np.array([LLst11, LLst12, LLst1, LLst2, LLst]), 1))

#真値での対数尤度
LLbest11 = np.sum(np.log(np.sum(thetat1[d_id21, ] * phit1.T[word1, ], axis=1)))
LLbest12 = np.sum(np.log(np.sum(thetat2[d_id22, ] * phit2.T[word2, ], axis=1)))
LLbest2 = np.sum(np.log(np.sum(gamma[x_id, ] * omegat.T[tag, ], axis=1)))
LLbest1 = LLbest11 + LLbest12
LLbest = LLbest1 + LLbest2
print(np.round(np.array([LLbest11, LLbest12, LLbest1, LLbest2, LLbest]), 1))

In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##単語トピックを生成
    #トピック選択確率を定義 
    Lho11 = theta1[d_id21, ] * phi1.T[word1, ]
    Lho12 = theta2[d_id22, ] * phi2.T[word2, ]
    topic_prob1 = Lho11 / np.dot(Lho11, k_vec)[:, np.newaxis]
    topic_prob2 = Lho12 / np.dot(Lho12, k_vec)[:, np.newaxis]

    #多項分布からトピックを生成
    Zi1 = np.array(rmnom(topic_prob1, f1, k, no1, 1)[1].todense(), dtype="int8")
    Zi2 = np.array(rmnom(topic_prob2, f2, k, no2, 1)[1].todense(), dtype="int8")


    ##ディリクレ分布からパラメータをサンプリング
    #グローバルトピック分布をサンプリング
    y1 = np.zeros((d, k), dtype="int")
    y2 = np.zeros((d, k), dtype="int")
    for i in range(d):
        z1 = Zi1[d_list11[i], ]; z2 = Zi2[d_list12[i], ]
        alpha1 = np.dot(d_dt11[i], z1); y1[i, ] = alpha1.copy()
        alpha2 = np.dot(d_dt12[i], z2); y2[i, ] = alpha2.copy()
        alpha = alpha1 + alpha2 + alpha01
        theta[i, ] = np.random.dirichlet(alpha, 1).reshape(-1)

    #ハイパーパラメータを更新
    delta_vec = np.dot(Delta1, k_vec)
    delta1 = psi(theta[s_id, ] + Delta1) - psi(Delta1)
    delta2 = (psi(delta_n1 + delta_vec) - psi(delta_vec))[:, np.newaxis]
    new_Delta1 = Delta1 * (delta1 / delta2) + er
    Delta1 = new_Delta1.copy()
    
    delta_vec = np.dot(Delta2, k_vec)
    delta1 = psi(theta[s_id, ] + Delta2) - psi(Delta2)
    delta2 = (psi(delta_n2 + delta_vec) - psi(delta_vec))[:, np.newaxis]
    new_Delta2 = Delta2 * (delta1 / delta2) + er
    Delta2 = new_Delta2.copy()
        
    #ローカルトピック分布をサンプリング
    for i in range(n):
        alpha1 = np.dot(d_dt21[i], Zi1[d_list21[i], ]) + new_Delta1[i, ]
        alpha2 = np.dot(d_dt22[i], Zi2[d_list22[i], ]) + new_Delta2[i, ]
        theta1[i, ] = np.random.dirichlet(alpha1, 1).reshape(-1)
        theta2[i, ] = np.random.dirichlet(alpha2, 1).reshape(-1)
    
        
    #単語分布をサンプリング
    x_freq1 = np.zeros((k, v1)); x_freq2 = np.zeros((k, v2))
    for i in range(v1):
        x_freq1[:, i] = np.dot(word_dt1[i], Zi1[word_list1[i], ]) 
        if i < v2:
            x_freq2[:, i] = np.dot(word_dt2[i], Zi2[word_list2[i], ]) 
    for j in range(k):
        phi1[j, ] = np.random.dirichlet(x_freq1[j, ] + beta01, 1).reshape(-1)
        phi2[j, ] = np.random.dirichlet(x_freq2[j, ] + beta01, 1).reshape(-1)


    ##タグトピックを生成
    #トピック選択確率を定義 
    theta_mu = y1 / w11[:, np.newaxis]
    Lho2 = theta_mu[x_id, ] * (omega.T)[tag, ]
    topic_prob = Lho2 / np.dot(Lho2, k_vec)[:, np.newaxis]

    #多項分布からトピックを生成
    Si = np.array(rmnom(topic_prob, g, k, np.arange(g), 1)[1].todense(), dtype="int8")

    ##ディリクレ分布からパラメータをサンプリング
    #タグ分布をサンプリング
    x_freq = np.zeros((k, a))
    for i in range(a):
        x_freq[:, i] = np.dot(tag_dt[i], Si[tag_list[i], ]) 
    for j in range(k):
        omega[j, ] = np.random.dirichlet(x_freq[j, ] + beta02, 1).reshape(-1)


    ##サンプリング結果の格納用と表示
    if rp%keep==0:
        #サンプリング結果を格納
        mkeep = int(rp/keep)
        THETA[:, :, mkeep] = theta
        THETA1[:, :, mkeep] = theta1
        THETA2[:, :, mkeep] = theta2
        PHI1[:, :, mkeep] = phi1
        PHI2[:, :, mkeep] =phi2
        OMEGA[:, :, mkeep] = omega
        
        #トピック割当はバーンイン期間を超えたら格納
        if (rp%keep==0) & (rp >= burnin):
            SEG11 = SEG11 + Zi1
            SEG12 = SEG12 + Zi2
            SEG2 = SEG2 + Si

    if rp%disp==0: 
        #対数尤度を更新
        LLho11 = np.sum(np.log(np.sum(Lho11, axis=1)))
        LLho12 = np.sum(np.log(np.sum(Lho12, axis=1)))
        LLho2 = np.sum(np.log(np.sum(Lho2, axis=1)))
        LLho1 = LLho11 + LLho12
        LLho = LLho1 + LLho2

        #サンプリング結果を表示
        print(rp)
        print(np.round([LLho11, LLho12, LLho1, LLho2, LLho], 1))
        print(np.round([LLst11, LLst12, LLst1, LLst2, LLst], 1))
        print(np.round([LLbest11, LLbest12, LLbest1, LLbest2, LLbest], 1))